# 🧪 Multi-Task Model Testing Notebook - UPDATED! 🔥

Notebook này dùng để test multi-task model (hierarchical classification + rotation prediction) trên test dataset.

## 📦 Model hiện tại:
**Conditional Model with Rotation Detection**
- Path: `models/conditional_model/1/best_model.pth`
- Architecture: ResNet50 + CBAM
- Tasks:
  - ✅ Hierarchical Classification (3 levels)
  - ✅ Rotation Detection (0°, 90°, 180°, 270°) - CHỈ cho SinoNom
- Training: Data augmentation x4 (mỗi ảnh SinoNom → 4 samples)
- Features:
  - Multi-GPU training
  - Mixed precision (AMP)
  - Frozen backbone + trainable CBAM & heads

---

## 🔄 Changelog - Latest Updates

### ✅ **FIXED MODEL ARCHITECTURE MISMATCH** (Oct 30, 2025) 🔧

**Problem**: RuntimeError - Model architecture không khớp với checkpoint!
- Test notebook dùng `MultiTaskResNet50` (simple CBAM)
- Checkpoint trained với `HierarchicalResNet50WithRotation` (advanced CBAM)

**Solution**: Cập nhật test notebook để sử dụng ĐÚNG architecture:

**1. CBAM Architecture** (✅ Updated):
```python
# OLD (Simple CBAM):
class ChannelAttention:
    fc1 = nn.Conv2d(...)  # ❌ Missing in checkpoint
    fc2 = nn.Conv2d(...)

# NEW (Training notebook CBAM):
class ChannelAttention:
    shared_mlp = nn.Sequential(  # ✅ Matches checkpoint
        nn.Flatten(),
        nn.Linear(...),
        nn.ReLU(),
        nn.Linear(...)
    )
```

**2. Model Architecture** (✅ Updated):
```python
# OLD:
class MultiTaskResNet50:
    fc_main = nn.Linear(...)  # ❌ Missing in checkpoint
    fc_doc = nn.Linear(...)
    fc_text = nn.Linear(...)
    fc_rotation = nn.Linear(...)

# NEW:
class HierarchicalResNet50WithRotation:
    h1_layer = nn.Sequential(...)  # ✅ Matches checkpoint
    h2_layer = nn.Sequential(...)
    h3_layer = nn.Sequential(...)
    classifier1 = nn.Linear(...)
    classifier2 = nn.Linear(...)
    classifier3 = nn.Linear(...)
    rotation_head = nn.Sequential(...)
```

**3. Output Format** (✅ Updated):
```python
# OLD:
outputs = [out_main, out_doc, out_text, out_rotation]

# NEW:
hierarchical_outputs, rotation_output = model(x)
# hierarchical_outputs = [out_main, out_doc, out_text]
# rotation_output = out_rotation
```

**Changes**:
1. **Model Architecture**: `HierarchicalResNet50WithRotation` (✅ matches training)
2. **CBAM Implementation**: Advanced CBAM with `shared_mlp` and `ChannelPool`
3. **Output Handling**: Tuple unpacking for hierarchical & rotation outputs
4. **Model Loading**: Handle DataParallel wrapped models
5. **Image Size**: 128x128 (unchanged)
6. **Rotation Testing**: Enabled by default (`test_rotations=True`)

**Expected Performance**:
- Main Category Acc: ~95%+
- Document Type Acc: ~98%+ (SinoNom only)
- Text Direction Acc: ~99%+ (Thong_thuong only)
- Rotation Acc: **80-90%** (SinoNom only, all 4 angles) ⬆️

---

## 📚 1. Import libraries & Define Constants

## ⚠️ Fix Numpy Version Issue

**Problem**: "Numpy is not available" error khi đọc ảnh
- Numpy 2.x không tương thích với Pillow cũ
- Cần downgrade về numpy < 2.0

**Solution**: Run cell dưới để install numpy đúng version

In [ ]:
# ⚠️ FIX: Install numpy < 2.0 to fix "Numpy is not available" error
# Numpy 2.x causes compatibility issues with Pillow when reading images
!pip install 'numpy<2' --upgrade

print("✅ Numpy version fixed! Please RESTART KERNEL and run cells again.")

  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.

In [ ]:
# ✅ Verify package versions
import numpy as np
from PIL import Image

print("="*60)
print("📦 Package Versions")
print("="*60)
print(f"Numpy version: {np.__version__}")
print(f"Pillow version: {Image.__version__}")

# Check numpy version
numpy_version = tuple(map(int, np.__version__.split('.')[:2]))
if numpy_version >= (2, 0):
    print("\n⚠️ WARNING: Numpy 2.x detected!")
    print("   Please run the cell above to downgrade to numpy < 2.0")
    print("   Then RESTART KERNEL and run cells again.")
else:
    print(f"\n✅ Numpy version OK: {np.__version__}")
print("="*60)

In [14]:
# ==================== IMPORT LIBRARIES ====================
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm import tqdm
import datetime
from typing import Dict, Tuple, List, Optional, Union
import io

# Excel export
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage
from openpyxl.styles import Alignment, PatternFill, Font
from openpyxl.utils import get_column_letter

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

# PyTorch
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# ==================== DEFINE CONSTANTS ====================

# Hierarchical class mappings
MAIN_CATEGORIES = {"SinoNom": 0, "NonSinoNom": 1}
DOC_TYPES = {"Thong_thuong": 0, "Hanh_chinh": 1, "Ngoai_canh": 2}
TEXT_DIRECTIONS = {"Doc": 0, "Ngang": 1}

# Rotation angles mapping
ROTATION_ANGLES = {0: 0, 90: 1, 180: 2, 270: 3}

# Reverse mappings for inference
INV_MAIN_CATEGORIES = {v: k for k, v in MAIN_CATEGORIES.items()}
INV_DOC_TYPES = {v: k for k, v in DOC_TYPES.items()}
INV_TEXT_DIRECTIONS = {v: k for k, v in TEXT_DIRECTIONS.items()}
INV_ROTATION_ANGLES = {v: k for k, v in ROTATION_ANGLES.items()}

# Model configuration
IMAGE_SIZE = (128, 128)  # ✅ UPDATED: 128x128 for conditional model (was 224x224)
BATCH_SIZE = 32
NUM_WORKERS = 0

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {DEVICE}")

# Paths
OUTPUT_PATH = "outputs"
TEST_PATH = "test_full"  # Thư mục chứa test data

print("✅ Đã import libraries và define constants thành công!")

PyTorch version: 2.2.2
CUDA available: False
🖥️ Using device: cpu
✅ Đã import libraries và define constants thành công!


## 🏗️ 2. Define CBAM Attention Mechanism & Multi-Task Model Architecture

**⚠️ UPDATED**: Model architecture phải KHỚP với training notebook!

**Model**: `HierarchicalResNet50WithRotation`
- Architecture: ResNet50 backbone + CBAM attention
- Features:
  - **Deep Hierarchical Classification**: h1_layer → h2_layer → h3_layer với feature concatenation
  - **Rotation Detection Head**: Separate branch từ base features
- Output format: `[hierarchical_outputs, rotation_output]`
  - `hierarchical_outputs = [out_main, out_doc, out_text]`
  - `rotation_output = out_rotation`

In [7]:
# ==================== CBAM ATTENTION MECHANISM ====================

class ChannelAttention(nn.Module):
    """Channel Attention in CBAM."""
    
    def __init__(self, channel_in, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelAttention, self).__init__()
        self.pool_types = pool_types
        
        self.shared_mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channel_in, channel_in // reduction_ratio),
            nn.ReLU(inplace=True),
            nn.Linear(channel_in // reduction_ratio, channel_in)
        )
    
    def forward(self, x):
        channel_attentions = []
        
        for pool_type in self.pool_types:
            if pool_type == 'avg':
                pool_init = nn.AvgPool2d(kernel_size=(x.size(2), x.size(3)), 
                                       stride=(x.size(2), x.size(3)))
                avg_pool = pool_init(x)
                channel_attentions.append(self.shared_mlp(avg_pool))
            elif pool_type == 'max':
                pool_init = nn.MaxPool2d(kernel_size=(x.size(2), x.size(3)), 
                                       stride=(x.size(2), x.size(3)))
                max_pool = pool_init(x)
                channel_attentions.append(self.shared_mlp(max_pool))
        
        pooling_sums = torch.stack(channel_attentions, dim=0).sum(dim=0)
        scaled = torch.sigmoid(pooling_sums).unsqueeze(2).unsqueeze(3).expand_as(x)
        
        return x * scaled

class ChannelPool(nn.Module):
    """Merge channels into 2 channels (max and mean)."""
    
    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1), 
                         torch.mean(x, 1).unsqueeze(1)), dim=1)

class SpatialAttention(nn.Module):
    """Spatial Attention in CBAM."""
    
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        
        self.compress = ChannelPool()
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=kernel_size, stride=1, 
                     padding=(kernel_size-1)//2, bias=False),
            nn.BatchNorm2d(1, eps=1e-5, momentum=0.01, affine=True)
        )
    
    def forward(self, x):
        x_compress = self.compress(x)
        x_output = self.spatial_attention(x_compress)
        scaled = torch.sigmoid(x_output)
        return x * scaled

class CBAM(nn.Module):
    """Convolutional Block Attention Module."""
    
    def __init__(self, channel_in, reduction_ratio=16, pool_types=['avg', 'max'], spatial=True):
        super(CBAM, self).__init__()
        self.spatial = spatial
        
        self.channel_attention = ChannelAttention(channel_in, reduction_ratio, pool_types)
        
        if self.spatial:
            self.spatial_attention = SpatialAttention(kernel_size=7)
    
    def forward(self, x):
        x_out = self.channel_attention(x)
        if self.spatial:
            x_out = self.spatial_attention(x_out)
        return x_out

print("✅ CBAM attention mechanism defined!")

# ==================== BOTTLENECK BLOCK & MULTI-TASK RESNET50 ====================

class BottleneckCBAM(nn.Module):
    """Bottleneck block with CBAM attention."""
    expansion = 4
    
    def __init__(self, inplanes, planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.use_cbam = use_cbam
        self.cbam = CBAM(planes * self.expansion) if use_cbam else None
    
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.cbam:
            out = self.cbam(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity
        out = self.relu(out)
        
        return out


class HierarchicalResNet50WithRotation(nn.Module):
    """
    ResNet50 with CBAM + Deep Hierarchical Classification + Rotation Detection.
    
    Multi-task learning:
    - Shared backbone for feature extraction
    - Hierarchical classification head (3 levels)
    - Rotation detection head (4 classes: 0°, 90°, 180°, 270°)
    """
    
    def __init__(self, use_cbam=True, image_depth=3, num_classes=[2, 3, 2],
                 enable_rotation=True, num_rotation_classes=4):
        super().__init__()
        self.expansion = 4
        self.use_cbam = use_cbam
        self.enable_rotation = enable_rotation
        self.inplanes = 64
        
        # ==================== SHARED BACKBONE ====================
        self.conv1 = nn.Conv2d(image_depth, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(BottleneckCBAM, 64, 3, use_cbam=use_cbam)
        self.layer2 = self._make_layer(BottleneckCBAM, 128, 4, stride=2, use_cbam=use_cbam)
        self.layer3 = self._make_layer(BottleneckCBAM, 256, 6, stride=2, use_cbam=use_cbam)
        self.layer4 = self._make_layer(BottleneckCBAM, 512, 3, stride=2, use_cbam=use_cbam)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # ==================== HIERARCHICAL CLASSIFICATION HEAD ====================
        base_feature_dim = 512 * self.expansion  # 2048
        
        # Level 1: Main category (SinoNom/NonSinoNom)
        self.h1_layer = nn.Sequential(
            nn.Linear(base_feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )
        
        # Level 2: Document type (depends on h1)
        self.h2_layer = nn.Sequential(
            nn.Linear(base_feature_dim + 512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4)
        )
        
        # Level 3: Text direction (depends on h1 + h2)
        self.h3_layer = nn.Sequential(
            nn.Linear(base_feature_dim + 512 + 256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        # Classification heads
        self.classifier1 = nn.Linear(512, num_classes[0])  # Main category
        self.classifier2 = nn.Linear(256, num_classes[1])  # Document type
        self.classifier3 = nn.Linear(128, num_classes[2])  # Text direction
        
        # ==================== ROTATION DETECTION HEAD ====================
        if self.enable_rotation:
            self.rotation_head = nn.Sequential(
                nn.Linear(base_feature_dim, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3),
                nn.Linear(256, num_rotation_classes)
            )
        
        self._initialize_weights()
    
    def _make_layer(self, block, planes, blocks, stride=1, use_cbam=True):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )
        
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, use_cbam=use_cbam))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, use_cbam=use_cbam))
        
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # ==================== SHARED BACKBONE ====================
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        base_features = torch.flatten(x, 1)  # [batch, 2048]
        
        # ==================== HIERARCHICAL CLASSIFICATION ====================
        # Level 1
        h1 = self.h1_layer(base_features)  # [batch, 512]
        out_main = self.classifier1(h1)
        
        # Level 2
        h2_input = torch.cat([base_features, h1], dim=1)
        h2 = self.h2_layer(h2_input)  # [batch, 256]
        out_doc = self.classifier2(h2)
        
        # Level 3
        h3_input = torch.cat([base_features, h1, h2], dim=1)
        h3 = self.h3_layer(h3_input)  # [batch, 128]
        out_text = self.classifier3(h3)
        
        # ==================== ROTATION DETECTION ====================
        if self.enable_rotation:
            out_rotation = self.rotation_head(base_features)
            return [out_main, out_doc, out_text], out_rotation
        else:
            return [out_main, out_doc, out_text]

print("✅ HierarchicalResNet50WithRotation with CBAM architecture defined!")

✅ CBAM attention mechanism defined!
✅ HierarchicalResNet50WithRotation with CBAM architecture defined!


## 🔧 3. Helper Functions

In [8]:
def load_multi_task_model(model_path, device=DEVICE):
    """
    Load trained multi-task model từ checkpoint.
    """
    print(f"📂 Loading model from {model_path}...")
    
    # Create model instance - sử dụng HierarchicalResNet50WithRotation với CBAM
    model = HierarchicalResNet50WithRotation(
        use_cbam=True,
        image_depth=3,
        num_classes=[2, 3, 2],  # [main_category, doc_type, text_direction]
        enable_rotation=True,
        num_rotation_classes=4
    )
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    
    # Load model state - handle both wrapped and unwrapped models
    if 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
    else:
        state_dict = checkpoint
    
    # Handle DataParallel wrapped models (remove 'module.' prefix)
    if list(state_dict.keys())[0].startswith('module.'):
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    
    model.load_state_dict(state_dict)
    
    model = model.to(device)
    model.eval()
    
    # Print model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"✅ Model loaded successfully!")
    print(f"   📊 Total parameters: {total_params:,}")
    print(f"   🏋️ Trainable parameters: {trainable_params:,}")
    
    return model

def rotate_image(image, angle):
    """
    Xoay ảnh theo góc chỉ định.
    """
    if angle == 0:
        return image
    elif angle == 90:
        return image.rotate(90, expand=True)
    elif angle == 180:
        return image.rotate(180, expand=True)
    elif angle == 270:
        return image.rotate(270, expand=True)
    else:
        return image

def filter_valid_labels(true_labels, pred_labels, valid_set):
    """
    Lọc các labels hợp lệ để tính confusion matrix.
    """
    filtered_true = []
    filtered_pred = []
    
    for t, p in zip(true_labels, pred_labels):
        if t in valid_set and p in valid_set:
            filtered_true.append(t)
            filtered_pred.append(p)
    
    return filtered_true, filtered_pred

print("✅ Helper functions defined!")

✅ Helper functions defined!


## 🧪 4. Main Evaluation Function

In [ ]:
def evaluate_multi_task_model(model_path, test_folder="test_full", output_path="outputs", test_rotations=False):
    """
    Đánh giá multi-task model trên test dataset.
    
    Args:
        model_path: Đường dẫn đến file checkpoint model
        test_folder: Thư mục chứa test data
        output_path: Thư mục lưu kết quả
        test_rotations: Nếu True, test model với các góc xoay khác nhau
    """
    print("🚀 Starting multi-task model evaluation...")
    
    # Load model
    model = load_multi_task_model(model_path)
    
    # Prepare transforms
    transform = transforms.Compose([
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create Excel workbook
    wb = Workbook()
    ws = wb.active
    ws.title = "Multi-Task Test Results"
    
    # Set up headers
    headers = [
        "Image", "File Name", 
        "Main Category", "Main Conf",
        "Doc Type", "Doc Conf",
        "Text Direction", "Text Conf",
        "Rotation (deg)", "Rotation Conf"
    ]
    
    header_font = Font(size=14, bold=True)
    cell_font = Font(size=12)
    
    for col, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col, value=header)
        cell.font = header_font
        cell.fill = PatternFill("solid", fgColor="CCCCCC")
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    # Set column widths
    ws.column_dimensions['A'].width = 60  # Image
    ws.column_dimensions['B'].width = 50  # Filename
    for col in range(3, len(headers) + 1):
        ws.column_dimensions[get_column_letter(col)].width = 18
    
    # Initialize lists for metrics
    true_main = []
    pred_main = []
    true_doc = []
    pred_doc = []
    true_text = []
    pred_text = []
    true_rotation = []
    pred_rotation = []
    
    # Collect test files
    test_files = []
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.avif', '.jfif', '.webp']
    
    for root, _, files in os.walk(test_folder):
        for file in files:
            if any(file.lower().endswith(ext) for ext in img_extensions):
                test_files.append(os.path.join(root, file))
    
    print(f"Found {len(test_files)} images to process...")
    
    # Determine rotation angles to test
    rotation_angles_to_test = [0, 90, 180, 270] if test_rotations else [0]
    
    # Process each image
    row = 2
    
    for img_path in tqdm(test_files, desc="Processing images"):
        try:
            # Get true labels from folder structure
            rel_path = os.path.relpath(img_path, test_folder)
            path_parts = rel_path.split(os.sep)
            
            # Extract true labels
            true_main_cat = path_parts[0]
            true_doc_type = path_parts[1] if len(path_parts) > 1 else "N/A"
            true_text_dir = path_parts[2] if len(path_parts) > 2 else "N/A"
            
            # Load original image - handle various formats
            try:
                img = Image.open(img_path)
                # Convert to RGB immediately to avoid numpy issues
                if img.mode != 'RGB':
                    img = img.convert('RGB')
            except Exception as img_error:
                print(f"⚠️ Cannot read image {img_path}: {img_error}")
                continue
            
            # Test with different rotation angles
            for true_rot_angle in rotation_angles_to_test:
                # Rotate image
                rotated_img = rotate_image(img.copy(), true_rot_angle)
                
                # Transform and predict
                tensor = transform(rotated_img).unsqueeze(0).to(DEVICE)
                
                with torch.no_grad():
                    # Model returns: [hierarchical_outputs, rotation_output]
                    # hierarchical_outputs = [out_main, out_doc, out_text]
                    hierarchical_outputs, rotation_output = model(tensor)
                    
                    # Process hierarchical classification outputs
                    hier_probs = [F.softmax(out, dim=1).squeeze(0).cpu().numpy() for out in hierarchical_outputs]
                    hier_preds = [int(np.argmax(p)) for p in hier_probs]
                    hier_confs = [float(np.max(p)) for p in hier_probs]
                    
                    # Process rotation output
                    rot_probs = F.softmax(rotation_output, dim=1).squeeze(0).cpu().numpy()
                    rot_pred = int(np.argmax(rot_probs))
                    rot_conf = float(np.max(rot_probs))
                
                # Convert predictions to labels
                pred_main_cat = INV_MAIN_CATEGORIES.get(hier_preds[0], str(hier_preds[0]))
                pred_doc_type = "N/A"
                pred_text_dir = "N/A"
                doc_conf = 1.0
                text_conf = 1.0
                
                if hier_preds[0] == MAIN_CATEGORIES["SinoNom"]:
                    pred_doc_type = INV_DOC_TYPES.get(hier_preds[1], str(hier_preds[1]))
                    doc_conf = hier_confs[1]
                    if hier_preds[1] == DOC_TYPES["Thong_thuong"]:
                        pred_text_dir = INV_TEXT_DIRECTIONS.get(hier_preds[2], str(hier_preds[2]))
                        text_conf = hier_confs[2]
                
                pred_rot_angle = INV_ROTATION_ANGLES.get(rot_pred, 0)
                
                # Add to confusion matrix lists
                true_main.append(true_main_cat)
                pred_main.append(pred_main_cat)
                
                if true_doc_type != "N/A" and pred_doc_type != "N/A":
                    true_doc.append(true_doc_type)
                    pred_doc.append(pred_doc_type)
                
                if true_text_dir != "N/A" and pred_text_dir != "N/A":
                    true_text.append(true_text_dir)
                    pred_text.append(pred_text_dir)
                
                true_rotation.append(str(true_rot_angle))
                pred_rotation.append(str(pred_rot_angle))
                
                # Add row to Excel
                try:
                    img_thumb = rotated_img.copy()
                    img_thumb.thumbnail((300, 300))
                    img_buffer = io.BytesIO()
                    # Save as PNG to avoid format issues
                    img_thumb.save(img_buffer, format='PNG')
                    xl_image = XLImage(img_buffer)
                    
                    ws.row_dimensions[row].height = 225
                    xl_image.anchor = f'A{row}'
                    ws.add_image(xl_image)
                except Exception as thumb_error:
                    print(f"⚠️ Cannot create thumbnail for {img_path}: {thumb_error}")
                    # Leave image cell empty if thumbnail fails
                
                # Add data
                filename = f"{os.path.basename(img_path)} (rot={true_rot_angle}°)"
                data_values = [
                    filename,
                    pred_main_cat, f"{hier_confs[0]:.3f}",
                    pred_doc_type, f"{doc_conf:.3f}",
                    pred_text_dir, f"{text_conf:.3f}",
                    f"{pred_rot_angle}°", f"{rot_conf:.3f}"
                ]
                
                for col, value in enumerate(data_values, 2):
                    cell = ws.cell(row=row, column=col, value=value)
                    cell.font = cell_font
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                
                row += 1
                
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
    
    # Filter valid labels for confusion matrices
    if true_doc:
        true_doc, pred_doc = filter_valid_labels(
            true_doc, pred_doc, 
            {"Ngoai_canh", "Thong_thuong", "Hanh_chinh"}
        )
    
    if true_text:
        true_text, pred_text = filter_valid_labels(
            true_text, pred_text,
            {"Doc", "Ngang"}
        )
    
    # Print classification reports
    print("\n" + "="*60)
    print("📊 CLASSIFICATION REPORTS")
    print("="*60)
    
    print("\n🏷️ Main Category:")
    print(classification_report(true_main, pred_main, digits=4))
    print(f"Accuracy: {accuracy_score(true_main, pred_main):.4f}")
    
    if true_doc:
        print("\n📋 Document Type:")
        print(classification_report(true_doc, pred_doc, digits=4))
        print(f"Accuracy: {accuracy_score(true_doc, pred_doc):.4f}")
    
    if true_text:
        print("\n📐 Text Direction:")
        print(classification_report(true_text, pred_text, digits=4))
        print(f"Accuracy: {accuracy_score(true_text, pred_text):.4f}")
    
    print("\n🔄 Rotation Angle:")
    print(classification_report(true_rotation, pred_rotation, digits=4))
    print(f"Accuracy: {accuracy_score(true_rotation, pred_rotation):.4f}")
    
    # Create confusion matrices
    num_plots = 4 if (true_doc and true_text) else 3 if true_doc else 2
    fig, axes = plt.subplots(1, num_plots, figsize=(6*num_plots, 5))
    
    if num_plots == 1:
        axes = [axes]
    
    plot_idx = 0
    
    # Main category
    cm_main = confusion_matrix(true_main, pred_main)
    sns.heatmap(cm_main, annot=True, fmt='d',
                xticklabels=sorted(set(true_main)),
                yticklabels=sorted(set(true_main)),
                cmap='Blues', ax=axes[plot_idx], square=True,
                cbar_kws={'label': 'Count'})
    axes[plot_idx].set_title('Main Category', fontsize=12, pad=10)
    axes[plot_idx].set_xlabel('Predicted', fontsize=10)
    axes[plot_idx].set_ylabel('True', fontsize=10)
    plot_idx += 1
    
    # Document type
    if true_doc:
        cm_doc = confusion_matrix(true_doc, pred_doc)
        sns.heatmap(cm_doc, annot=True, fmt='d',
                    xticklabels=sorted(set(true_doc)),
                    yticklabels=sorted(set(true_doc)),
                    cmap='Blues', ax=axes[plot_idx], square=True,
                    cbar_kws={'label': 'Count'})
        axes[plot_idx].set_title('Document Type', fontsize=12, pad=10)
        axes[plot_idx].set_xlabel('Predicted', fontsize=10)
        axes[plot_idx].set_ylabel('True', fontsize=10)
        plot_idx += 1
    
    # Text direction
    if true_text:
        cm_text = confusion_matrix(true_text, pred_text)
        sns.heatmap(cm_text, annot=True, fmt='d',
                    xticklabels=sorted(set(true_text)),
                    yticklabels=sorted(set(true_text)),
                    cmap='Blues', ax=axes[plot_idx], square=True,
                    cbar_kws={'label': 'Count'})
        axes[plot_idx].set_title('Text Direction', fontsize=12, pad=10)
        axes[plot_idx].set_xlabel('Predicted', fontsize=10)
        axes[plot_idx].set_ylabel('True', fontsize=10)
        plot_idx += 1
    
    # Rotation
    cm_rotation = confusion_matrix(true_rotation, pred_rotation)
    sns.heatmap(cm_rotation, annot=True, fmt='d',
                xticklabels=sorted(set(true_rotation), key=lambda x: int(x)),
                yticklabels=sorted(set(true_rotation), key=lambda x: int(x)),
                cmap='Blues', ax=axes[plot_idx], square=True,
                cbar_kws={'label': 'Count'})
    axes[plot_idx].set_title('Rotation Angle', fontsize=12, pad=10)
    axes[plot_idx].set_xlabel('Predicted', fontsize=10)
    axes[plot_idx].set_ylabel('True', fontsize=10)
    
    plt.tight_layout(pad=3.0)
    
    # Save results
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    os.makedirs(output_path, exist_ok=True)
    
    # Save confusion matrices
    plt.savefig(os.path.join(output_path, f'multi_task_confusion_matrices_{timestamp}.png'),
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    # Save Excel file
    excel_path = os.path.join(output_path, f'multi_task_test_results_{timestamp}.xlsx')
    wb.save(excel_path)
    
    print(f"\n✅ Evaluation complete!")
    print(f"📊 Results saved to {excel_path}")
    print(f"📈 Confusion matrices saved to {output_path}/multi_task_confusion_matrices_{timestamp}.png")

print("✅ Evaluation function defined!")

✅ Evaluation function defined!


## 🚀 5. Run Evaluation - Conditional Model

**Model**: `models/conditional_model/1/best_model.pth`

**Cấu hình**:
- ✅ `test_rotations=True`: Test với TẤT CẢ 4 góc xoay (0°, 90°, 180°, 270°)
- Dataset: `test_full/` 
- Output: Excel file với predictions và confidence scores

**Kết quả sẽ bao gồm**:
1. Main Category (SinoNom/NonSinoNom)
2. Document Type (Thong_thuong/Hanh_chinh/Ngoai_canh) - CHỈ cho SinoNom
3. Text Direction (Doc/Ngang) - CHỈ cho Thong_thuong
4. Rotation Angle (0°/90°/180°/270°) - CHỈ cho SinoNom

Chạy đánh giá model trên test dataset.

In [15]:
# ✅ Verify model path và dataset
import os

MODEL_PATH = "models/conditional_model/1/best_model.pth"

print("="*70)
print("🔍 Verification")
print("="*70)

# Check model file
if os.path.exists(MODEL_PATH):
    model_size = os.path.getsize(MODEL_PATH) / (1024**2)  # Convert to MB
    print(f"✅ Model file found: {MODEL_PATH}")
    print(f"   Size: {model_size:.2f} MB")
else:
    print(f"❌ Model NOT found: {MODEL_PATH}")
    print("   Available models:")
    if os.path.exists("models/conditional_model/1/"):
        for f in os.listdir("models/conditional_model/1/"):
            print(f"   - {f}")

# Check test dataset
if os.path.exists(TEST_PATH):
    print(f"\n✅ Test dataset found: {TEST_PATH}")
    # Count test images
    total_images = 0
    for root, dirs, files in os.walk(TEST_PATH):
        total_images += len([f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.avif', '.jfif'))])
    print(f"   Total test images: {total_images}")
else:
    print(f"\n❌ Test dataset NOT found: {TEST_PATH}")

# Check output directory
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print(f"\n📁 Created output directory: {OUTPUT_PATH}")
else:
    print(f"\n✅ Output directory exists: {OUTPUT_PATH}")

print("="*70)

🔍 Verification
✅ Model file found: models/conditional_model/1/best_model.pth
   Size: 156.39 MB

✅ Test dataset found: test_full
   Total test images: 625

✅ Output directory exists: outputs


In [16]:
# ✅ UPDATED: Đường dẫn đến model checkpoint MỚI
MODEL_PATH = "models/conditional_model/1/best_model.pth"  # Conditional model with rotation detection

# Chạy evaluation
# test_rotations=False: chỉ test với góc xoay 0 độ (ảnh gốc)
# test_rotations=True: test với các góc xoay 0, 90, 180, 270 độ
evaluate_multi_task_model(
    model_path=MODEL_PATH,
    test_folder=TEST_PATH,
    output_path=OUTPUT_PATH,
    test_rotations=True  # ✅ Enable rotation testing cho conditional model
)

🚀 Starting multi-task model evaluation...
📂 Loading model from models/conditional_model/1/best_model.pth...
✅ Model loaded successfully!
   📊 Total parameters: 29,292,731
   🏋️ Trainable parameters: 29,292,731
Found 645 images to process...
✅ Model loaded successfully!
   📊 Total parameters: 29,292,731
   🏋️ Trainable parameters: 29,292,731
Found 645 images to process...


Processing images:   1%|          | 7/645 [00:00<00:10, 61.01it/s]

Error processing test_full/NonSinoNom/non_sino_nom_021.webp: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_032.jpeg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_033.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_027.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_026.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_049.jpeg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_030.png: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_024.png: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_025.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_019.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_019.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_031.png: Numpy is not available
E

Processing images:   3%|▎         | 17/645 [00:01<01:03,  9.91it/s]

Error processing test_full/NonSinoNom/non_sino_nom_009.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_008.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_020.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_022.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_023.jpg: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_044.png: Numpy is not available
Error processing test_full/NonSinoNom/non_sino_nom_043.jpeg: Numpy is not available


...

Processing images:  30%|██▉       | 193/645 [00:10<00:37, 11.97it/s]

Error processing test_full/SinoNom/Thong_thuong/Ngang/ngang_582.jpg: Numpy is not available
Error processing test_full/SinoNom/Thong_thuong/Ngang/ngang_958.png: Numpy is not available
Error processing test_full/SinoNom/Thong_thuong/Ngang/ngang_1102.jpg: Numpy is not available


Processing images:  31%|███       | 200/645 [00:10<00:23, 19.25it/s]

Error processing test_full/SinoNom/Thong_thuong/Ngang/ngang_1314.jpg: Numpy is not available
Error processing test_full/SinoNom/Thong_thuong/Ngang/ngang_409.png: Numpy is not available
Error processing test_full/SinoNom/Thong_thuong/Ngang/thong_thuong_ngang_071.png: Numpy is not available
Error processing test_full/SinoNom/Thong_thuong/Ngang/ngang_390.png: Numpy is not available
Error processing test_full/SinoNom/Thong_thuong/Ngang/thong_thuong_ngang_070.png: Numpy is not available


KeyboardInterrupt: 

## 📝 6. Notes

### Cấu trúc test folder:
```
test_full/
├── SinoNom/
│   ├── Thong_thuong/
│   │   ├── Doc/
│   │   └── Ngang/
│   ├── Hanh_chinh/
│   └── Ngoai_canh/
└── NonSinoNom/
```

### Output files:
- **Excel file**: Chi tiết kết quả dự đoán cho từng ảnh với thumbnail
- **Confusion matrices**: Visualization của confusion matrices cho tất cả các tasks

### Model tasks:
1. **Main Category**: SinoNom vs NonSinoNom
2. **Document Type**: Thong_thuong, Hanh_chinh, Ngoai_canh (chỉ cho SinoNom)
3. **Text Direction**: Doc, Ngang (chỉ cho Thong_thuong)
4. **Rotation**: 0°, 90°, 180°, 270°